# Lab Exercise 2: Unconstrained Least Squares Unmixing

## Objectives
- Implement the analytical solution for unconstrained least squares
- Apply it to hyperspectral data
- Evaluate reconstruction quality
- Understand limitations of unconstrained approach

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../src')

from data_loader import HyperspectralDataLoader
from visualization import HSIVisualizer
from optimization import HyperspectralUnmixer
from metrics import UnmixingEvaluator

%matplotlib inline

## Task 1: Load Data and Endmembers

Load the previously processed data and endmembers.

In [ ]:
# Load data
loader = HyperspectralDataLoader("../data/")
hsi_data, ground_truth = loader.load_indian_pines()
Y, _ = loader.vectorize_data()

# Load endmembers from previous exercise
try:
    S = np.load('../data/extracted_endmembers.npy')
    endmember_names = np.load('../data/endmember_names.npy', allow_pickle=True)
    print(f"Loaded endmembers: {S.shape}")
    print(f"Endmember names: {list(endmember_names)}")
except FileNotFoundError:
    print("Endmembers not found. Re-extracting...")
    S, endmember_names = loader.extract_class_spectra([1, 2, 3])

print(f"Data matrix Y shape: {Y.shape}")
print(f"Endmember matrix S shape: {S.shape}")

## Task 2: Implement Unconstrained Least Squares

**Mathematical Background:**

The unconstrained least squares problem is:
$$\min_{\mathbf{A}} \|\mathbf{S}\mathbf{A} - \mathbf{Y}\|_F^2$$

The analytical solution is:
$$\mathbf{A}^* = (\mathbf{S}^T\mathbf{S})^{-1}\mathbf{S}^T\mathbf{Y}$$

In [ ]:
def unconstrained_least_squares_manual(S, Y):
    """
    Manual implementation of unconstrained least squares.
    
    TODO: Implement the analytical solution
    """
    # Step 1: Compute S^T S
    StS = # YOUR CODE HERE
    
    # Step 2: Compute S^T Y  
    StY = # YOUR CODE HERE
    
    # Step 3: Solve (S^T S) A = S^T Y
    try:
        A = # YOUR CODE HERE (use np.linalg.solve)
    except np.linalg.LinAlgError:
        print("Warning: Using pseudo-inverse due to singular matrix")
        A = # YOUR CODE HERE (use np.linalg.pinv)
    
    return A

# Test your implementation
A_manual = unconstrained_least_squares_manual(S, Y)
print(f"Abundance matrix shape: {A_manual.shape}")

## Task 3: Compare with Library Implementation

Compare your manual implementation with the provided class method.

In [ ]:
# Use the provided implementation
unmixer = HyperspectralUnmixer()
A_library = unmixer.unconstrained_least_squares(S, Y)

# Compare results
difference = np.max(np.abs(A_manual - A_library))
print(f"Maximum difference between implementations: {difference:.2e}")

if difference < 1e-10:
    print("✓ Implementations match!")
else:
    print("✗ Implementations differ - check your code")

## Task 4: Analyze Abundance Properties

Examine the properties of the unconstrained abundances.

In [ ]:
# TODO: Analyze abundance statistics
evaluator = UnmixingEvaluator()
abundance_stats = # YOUR CODE HERE

print("Abundance Statistics:")
for key, value in abundance_stats.items():
    print(f"  {key}: {value:.6f}")

# Check for physical violations
print(f"\nPhysical Constraint Analysis:")
print(f"  Minimum abundance: {np.min(A_library):.4f}")
print(f"  Maximum abundance: {np.max(A_library):.4f}")
print(f"  Fraction of negative values: {abundance_stats['fraction_negative']:.4f}")

# Check sum-to-one constraint
abundance_sums = np.sum(A_library, axis=0)
print(f"  Mean sum of abundances: {np.mean(abundance_sums):.4f}")
print(f"  Std of abundance sums: {np.std(abundance_sums):.4f}")

## Task 5: Visualize Results

Create visualizations to understand the unmixing results.

In [ ]:
# TODO: Plot abundance maps
visualizer = HSIVisualizer()
height, width = hsi_data.shape[:2]

# YOUR CODE HERE to plot abundance maps

# TODO: Plot abundance statistics
# YOUR CODE HERE

## Task 6: Evaluate Reconstruction Quality

Compute reconstruction metrics to assess unmixing quality.

In [ ]:
# TODO: Evaluate reconstruction
rgb_bands = loader.get_rgb_bands()
results = # YOUR CODE HERE

print("Reconstruction Quality Metrics:")
print(f"  Mean SAM (degrees): {results['mean_sam_degrees']:.4f}")
print(f"  RMSE: {results['rmse']:.6f}")
if 'ssim_mean' in results:
    print(f"  Mean SSIM: {results['ssim_mean']:.4f}")
print(f"  SNR (dB): {results['snr_db']:.2f}")

## Task 7: Visualize Reconstruction

Compare original and reconstructed RGB composites.

In [ ]:
# TODO: Reconstruct hyperspectral data
Y_reconstructed = # YOUR CODE HERE
hsi_reconstructed = Y_reconstructed.T.reshape(height, width, -1)

# TODO: Plot comparison
# YOUR CODE HERE

## Task 8: Test with Synthetic Data

Validate the algorithm with synthetic data where ground truth is known.

In [ ]:
from data_loader import create_synthetic_data

# Create synthetic data
hsi_synthetic, S_true, A_true = create_synthetic_data(
    height=30, width=30, bands=50, num_endmembers=3, noise_level=0.05)

Y_synthetic = hsi_synthetic.reshape(-1, hsi_synthetic.shape[2]).T

# TODO: Apply unconstrained unmixing
A_estimated = # YOUR CODE HERE

# TODO: Compute abundance angle distance
aad = # YOUR CODE HERE

print(f"Synthetic Data Test:")
print(f"  Abundance Angle Distance: {np.degrees(aad):.4f}°")
print(f"  Reconstruction RMSE: {np.sqrt(np.mean((S_true @ A_estimated - Y_synthetic)**2)):.6f}")

# Visualize true vs estimated abundances
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for i in range(3):
    # True abundances
    axes[0, i].imshow(A_true[i, :].reshape(30, 30), cmap='hot', vmin=0, vmax=1)
    axes[0, i].set_title(f'True EM{i+1}')
    axes[0, i].axis('off')
    
    # Estimated abundances
    axes[1, i].imshow(A_estimated[i, :].reshape(30, 30), cmap='hot')
    axes[1, i].set_title(f'Estimated EM{i+1}')
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()

## Reflection Questions

1. **What problems do you observe with the unconstrained abundances?**

   *Your answer here*

2. **Why are negative abundances physically meaningless?**

   *Your answer here*

3. **Why is the sum-to-one constraint important for abundance interpretation?**

   *Your answer here*

4. **How does noise affect the unconstrained solution?**

   *Your answer here*

In [ ]:
# Save results for comparison in next notebook
np.save('../data/unconstrained_abundances.npy', A_library)
print("Results saved for comparison!")